In [ ]:
import climetlab as cml
import os
!pip install climetlab climetlab_maelstrom_downscaling

In [1]:
from tensorflow.keras.layers import (Input, Concatenate,Conv3D,LeakyReLU, Dense, Conv2D,Activation, BatchNormalization,
                                     Conv2DTranspose, Input, MaxPool2D)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import sigmoid, linear
from tensorflow.keras.preprocessing.image import NumpyArrayIterator
import tensorflow as tf
import tensorflow.python.ops.numpy_ops.np_config as np_config
tf.config.experimental_run_functions_eagerly(True)
np_config.enable_numpy_behavior()
import time
import keras
import tensorflow as tf
import keras.backend as K
from typing import Protocol
import time 

2022-05-12 15:40:11.408239: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [ ]:
cmlds_train = cml.load_dataset("maelstrom-downscaling", dataset="training")
cmlds_val = cml.load_dataset("maelstrom-downscaling", dataset="validation")
cmlds_test = cml.load_dataset("maelstrom-downscaling", dataset="testing")
ds_train.to_netcdf("ds_train.nc")
ds_val.to_netcdf("ds_val.nc")
ds_test.to_netcdf("ds_test.nc")

In [2]:
print(cmlds_train.all_datelist)
ds_train = cmlds_train.to_xarray()
ds_val = cmlds_val.to_xarray()
ds_test = cmlds_test.to_xarray()

NameError: name 'cmlds_train' is not defined

In [2]:
import xarray as xr
ds_train = xr.open_dataset("ds_train.nc")
ds_val = xr.open_dataset("ds_val.nc")
ds_test = xr.open_dataset("ds_test.nc")

In [3]:
import numpy as np
def reshape_ds(ds):
    ds = ds.to_array(dim = "variables").squeeze()
    ds = np.squeeze(ds.values)
    ds = np.transpose(ds, (1, 2, 3, 0))
    return ds
ds_test = reshape_ds(ds_test)
ds_val = reshape_ds(ds_val)
ds_train = reshape_ds(ds_train)

In [4]:
n_samples = ds_train.shape[0]

In [5]:
n_samples

1464

In [6]:
ds_train.shape

(1464, 96, 128, 4)

In [7]:
input_shape = [ds_train.shape[1],ds_train.shape[2], 2]
target_shape = [ds_train.shape[1],ds_train.shape[2],1]

In [8]:
input_shape

[96, 128, 2]

In [18]:
#auxiliary functions used for parsing the hyerparameters from hparams_dict
def reduce_dict(dict_in: dict, dict_ref: dict):
    """
    Reduces input dictionary to keys from reference dictionary. If the input dictionary lacks some keys, these are 
    copied over from the reference dictionary, i.e. the reference dictionary provides the defaults
    :param dict_in: input dictionary
    :param dict_ref: reference dictionary
    :return: reduced form of input dictionary (with keys complemented from dict_ref if necessary)
    """
    method = reduce_dict.__name__

    # sanity checks
    assert isinstance(dict_in, dict), "%{0}: dict_in must be a dictionary, but is of type {1}"\
                                      .format(method, type(dict_in))
    assert isinstance(dict_ref, dict), "%{0}: dict_ref must be a dictionary, but is of type {1}"\
                                       .format(method, type(dict_ref)) 

    dict_merged = {**dict_ref, **dict_in}
    dict_reduced = {key: dict_merged[key] for key in dict_ref}

    return dict_reduced

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [19]:

def conv_block(inputs: tf.Tensor = None, num_filters: int = None, kernel: tuple = (3,3), padding: str="same",
              activation: str = "relu", kernel_init: str = "he_normal", l_batch_normalization: bool=False): 

    """
    A convolutional layer with optional batch normalization
    :param inputs: the input data with dimensions nx, ny and nc
    :param num_filters: number of filters (output channel dimension)
    :param kernel: tuple indictating kernel size
    :param padding: technique for padding (e.g. "same" or "valid")
    :param activation: activation fuction for neurons (e.g. "relu")
    :param kernel_init: initialization technique (e.g. "he_normal" or "glorot_uniform")
    """
    x = Conv2D(num_filters, kernel, padding=padding, kernel_initializer=kernel_init)(inputs)
    if l_batch_normalization:
        x = BatchNormalization()(x)
    x = Activation(activation)(x)
    return x

def conv_block_n(inputs, num_filters, n=2, kernel=(3,3), padding="same", activation="relu", 
                     kernel_init="he_normal", l_batch_normalization=False):
    """
    Sequential application of two convolutional layers (using conv_block).
    """

    x = conv_block(inputs, num_filters, kernel, padding, activation,
                   kernel_init, l_batch_normalization)
    
    for i in np.arange(n-1):
        x = conv_block(x, num_filters, kernel, padding, activation,
                       kernel_init, l_batch_normalization)
    
    return x


def encoder_block(inputs, num_filters, kernel_maxpool: tuple=(2,2), l_large: bool=False):
    """
    One complete encoder-block used in U-net
    """
    if l_large:
        x = conv_block_n(inputs, num_filters, n=2)
    else:
        x = conv_block(inputs, num_filters)

    p = MaxPool2D(kernel_maxpool)(x)

    return x, p


def decoder_block(inputs, skip_features, num_filters, kernel: tuple=(3,3), strides_up: int=2, padding: str= "same",
                  activation: str="relu", kernel_init: str="he_normal", l_batch_normalization: bool=False):
    """
    One complete decoder block used in U-net (reverting the encoder)
    """

    x = Conv2DTranspose(num_filters, (strides_up, strides_up), strides=strides_up, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block_n(x, num_filters, 2, kernel, padding, activation, kernel_init, l_batch_normalization)
    return x

def generator(input_shape, channels_start=56, z_branch=False):
    """
    Function to build up the generator architecture, here we take UNET as generator
    """

    inputs = Input(input_shape)

    """ encoder """
    s1, e1 = encoder_block(inputs, channels_start, l_large=True)
    s2, e2 = encoder_block(e1, channels_start*2, l_large=False)
    s3, e3 = encoder_block(e2, channels_start*4, l_large=False)

    """ bridge encoder <-> decoder """
    b1 = conv_block(e3, channels_start*8)

    """ decoder """
    d1 = decoder_block(b1, s3, channels_start*4)
    d2 = decoder_block(d1, s2, channels_start*2)
    d3 = decoder_block(d2, s1, channels_start)

    output_temp = Conv2D(1, (1,1), kernel_initializer="he_normal", name="output_temp")(d3)
    if z_branch:
        output_z = Conv2D(1, (1, 1), kernel_initializer="he_normal", name="output_z")(d3)

        model = Model(inputs, [output_temp, output_z], name="t2m_downscaling_unet_with_z")
    else:    
        model = Model(inputs, output_temp, name="t2m_downscaling_unet")

    return model



In [20]:
def discriminator(target_shape):
    """
    Discriminator: this discriminator so far perfoms best on the precipitation dataset
    """

    x = Input(target_shape)
    conv1 = Conv2D(filters=4, kernel_size=2, strides=(1, 1), padding='same')(x)
   
    conv1 = Activation("relu")(conv1)
    conv2 = tf.reshape(conv1, [-1,1])
    fc2 = LeakyReLU(0.2)(conv2)
    out_logit = Dense(1)(fc2)
    out = tf.nn.sigmoid(out_logit) 
    D = Model(x, out)
    return D

def ciritic(target_shape):
    print("You are trainaing Wasserstain GAN")
    x = Input(target_shape)
    conv1 = Conv3D(filters=4, kernel_size=2, strides=(1, 1,1), padding='same')(x)
    conv1 = Activation("relu")(conv1)
    conv2 = tf.reshape(conv1, [-1,1])
    fc2 = LeakyReLU(0.2)(conv2)
    out = Dense(1, activiation="linear")(fc2)
    D = Model(x, out)
    return D

In [27]:
#This chunk defines the losses
from typing import Any, Callable
### Vanilla GAN loss        
def gan_gen_loss(D_fake):
    """
    Define generator loss

    Return:  the loss of generator given inputs
    """
    real_labels = tf.ones_like(D_fake)
    G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake, labels=real_labels))            
    return G_loss

def gan_disc_loss(D_real, D_fake):
    """
    Return the loss of discriminator (Vanilla GAN)
    """
    real_labels = tf.ones_like(D_real)
    gen_labels = tf.zeros_like(D_fake)
    D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real,
                                                                          labels=real_labels))
    D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake,
                                                                          labels=gen_labels))
    D_loss = D_loss_real + D_loss_fake
    return D_loss

def get_gan_loss(D_real, D_fake):
    D_loss = gan_disc_loss(D_real, D_fake)
    G_loss = gan_gen_loss(D_fake)
    return D_loss, G_loss

####WGAN loss
def wgan_gen_loss(D_fake):
    G_loss = -tf.reduce_mean(D_fake)
    return G_loss

def wgan_critic_loss(D_real,D_fake):
    """
    Return the loss of critic (WGAN)
    """
    D_loss = tf.reduce_mean(D_real) - tf.reduce_mean(D_fake)
    return D_loss

def get_wgan_losses(D_real, D_fake):
    G_loss =  wgan_gen_loss(D_fake)
    D_loss = wgan_critic_loss(D_real,D_fake)
    return D_loss, G_loss

## Reconstruction loss for generator (UNet)
def get_recon_loss(target, gen_images):
    """
    Get reconstruction loss 
    """
    recon_loss = tf.reduce_mean(tf.square(target - gen_images))
    return recon_loss

In [133]:
#define your hparameters in a dictionary
hparams_dict = {
    "batch_size": 4,
    "lr": 0.001,
    "max_epochs": 5,
    "context_frames": 7,
    "sequence_length": 15,
    "ngf": 16,
    "enable_gan": True, #enable gan
    "enable_wgan":True, #enable wgan 
    "enable_embed":False,  #enable the conditional information 
    "d_steps": 5, # The original paper recommends training, the discriminator for `x` more steps (typically 5) as compared t # one step of the generator.
    "alpha":0.00005, #WGAN hparams
    "clip_const":0.01, # default WGAN hparams
    "m":64, ##WGAN hparams
    "weight_recon":0.1
}

In [148]:
class WGANModel(object):

    def __init__(self, mode: str = "train", hparams_dict: dict = None, 
                 target_shape:list=None, input_shape:list=None,
                 discriminator: Callable=None, generator:Callable=None): 
        """
         This is a class for building convLSTM GAN architecture by using updated hparameters
             mode                  : string, either "train" or "val" 
             hparams_dict          : dictionary, contains the hyperparameters names and default values
             input_shape           : tf.Tensor shape equal to the input shape
        """
        self.generator = generator
        self.discriminator = discriminator
        self.mode = mode
        self.input_shape = input_shape
        self.target_shape = target_shape
        #obtain the hyperparmeters
        self.hparams_dict = hparams_dict
        self.hparams = self.parse_hparams()
        self.batch_size = self.hparams.batch_size
        self.learning_rate = self.hparams.lr
        self.max_epochs = self.hparams.max_epochs
        self.sequence_length = self.hparams.sequence_length
        self.context_frames = self.hparams.context_frames
        self.loss_fun = self.hparams.loss_fun
        self.enable_gan = self.hparams.enable_gan
        self.enable_wgan = self.hparams.enable_wgan
        self.enable_embed = self.hparams.enable_embed
        self.d_steps = self.hparams.d_steps
        self.ngf = self.hparams.ngf #latent dim
        self.weight_recon = self.hparams.weight_recon
        self.clip_const = self.hparams.clip_const
        #Class attributes
        self.recon_loss = None
        self.G_loss = None
        self.D_loss = None

        

    def hparams_check(self):
        if not self.enable_gan:
            if self.enable_wgan:
                raise("You must set enable_gan to 'True' in hparams_dict congifuration")

                
    def parse_hparams(self): 
        self.hparams_dict = dotdict(self.hparams_dict)
        return self.hparams_dict

    def define_optimizers(self):
        self.d_optim = tf.keras.optimizers.Adam(self.learning_rate)
        self.g_optim = tf.keras.optimizers.Adam(self.learning_rate)

    def get_losses(self, D_real, D_fake, target, gen_image):
        """
        Get the losses based on the adopted model (GAN, WGAN, or UNet)
        """
        # Reconstruction loss
        recon_loss = get_recon_loss(target,gen_image)

        if self.enable_wgan:
            G_loss, D_loss = get_wgan_losses(D_real, D_fake)
        else:
            #vanilla GAN
             G_loss, D_loss = get_gan_losses(D_real, D_fake)

        return G_loss, D_loss, recon_loss
    
    @tf.function
    def train_step(self, inputs, target, i, g_per_d):
        """
        Training model per step
        inputs: inputs tensorflow
        target: target tensor
        i: the training step
        g_per_d: the original paper recommends training, the discriminator for `x` more steps (typically 5) as compared t # one step of the generator.
        
        # For each batch, we are going to perform the
        # following steps as laid out in the original paper:
        # 1. Train the generator and get the generator loss
        # 2. Train the discriminator and get the discriminator loss
        
        # Train the discriminator first. The original paper recommends training
        # the discriminator for `x` more steps (typically 5) as compared to
        # one step of the generator.
        """

        self.G = self.generator(self.input_shape)
        self.D = self.discriminator(self.target_shape)
        self.define_optimizers()


        #Train discriminator/critic
        for step in range(self.d_steps):
            if self.enable_gan:
                with tf.GradientTape() as d_tape:
                    # Generate fake images given input images
                    gen_images = self.G(inputs)
                    # Get the logits for the real images
                    D_real = self.D(target)
                    # Get the logits for the fake images
                    D_fake = self.D(gen_images)
                    # Calculate the discriminator loss using the fake and real image logits
                    if self.enable_wgan:
                        if i == 0 and step == 0 :
                            print("You are training both generator and discriminator (WGAN)")
                            print("You are training {} times more discriminator/critic for one time generator".format(self.d_steps))   
                        D_loss = wgan_critic_loss(D_real,D_fake)
                    else:
                        if i == 0 and step == 0:
                            print("You are training both generator and discriminator (GAN)")
                            print("You are training {} times more discriminator/critic for one time generator".format(self.d_steps))   
                        D_loss = gan_disc_loss(D_real, D_fake)

                    d_gradients = d_tape.gradient(D_loss, self.D.trainable_variables)
                    #define the training stratigy (ratio of number iteration of training on discriminator)
                    self.d_optim.apply_gradients(zip(d_gradients, self.D.trainable_variables))


        # Train generator

        with tf.GradientTape() as g_tape:
            # Generate fake images given input images
            gen_images = self.G(inputs)
            D_fake = self.D(gen_images)
            G_loss = gan_gen_loss(D_fake)
            recon_loss = get_recon_loss(target, gen_images)
            total_gen_loss = (1-self.weight_recon) * G_loss + self.weight_recon*(recon_loss)
            g_gradients = g_tape.gradient(total_gen_loss, self.G.trainable_variables)
            self.g_optim.apply_gradients(zip(g_gradients, self.G.trainable_variables))
            for w in self.D.trainable_variables:
                w.assign(tf.clip_by_value(w, -self.clip_const, self.clip_const))
                
            
        return G_loss, D_loss, recon_loss


    def make_data_generator(self, ds_train,ds_val,ds_test):
        
        train_dataset = tf.data.Dataset.from_tensor_slices((ds_train[...,:2],ds_train[...,2:3]))
        val_dataset = tf.data.Dataset.from_tensor_slices((ds_val[...,:2],ds_val[...,2:3]))
        test_dataset = tf.data.Dataset.from_tensor_slices((ds_test[...,:2],ds_test[...,2:3]))
        train_dataset = train_dataset.shuffle(100).repeat(self.max_epochs).batch(self.batch_size)
        val_dataset = val_dataset.batch(self.batch_size)
        test_dataset = test_dataset.batch(self.batch_size)
        self.train_iterator = iter(train_dataset)
        self.val_iterator = iter(val_dataset) 
        self.test_iterator = iter(test_dataset) 
        
    def minic_train(self,n_samples,log_freq=10):
        
        iterations_epoch = n_samples // self.batch_size
        iteration = self.max_epochs * iterations_epoch

        for step in range(iteration):
            x,y  = next(self.train_iterator)
            train_start_time = time.time()
            time.sleep(0.1)
            train_step_time = time.time() - train_start_time

            if step % log_freq == 0:
                template = 'training time per step: {:.5f}/s'
                print(template.format(train_step_time))
                print(x.shape)


    def train(self,  n_samples,log_freq=5):
    
        iterations_epoch = n_samples // self.batch_size
        iteration = self.max_epochs * iterations_epoch

        for step in range(iteration):
            x,y = next(self.train_iterator)

            train_start_time = time.time()

            g_loss, d_loss, recon_loss = self.train_step(x, y, step, self.d_steps)
            train_step_time = time.time() - train_start_time

            if step % log_freq == 0:
                template = '[{}/{}] D_loss={:.5f} G_loss={:.5f}, g_recon_loss={:.5f} training time per step: {:.5f}/s'
                print(template.format(step, iteration, d_loss, g_loss,recon_loss, train_step_time))

                
    # def prediction(self):
    #     iterations = self.val_samples // self.batch_size
    #     (x_val,y_val) = next(self.val_iterator)
    #     is_first = True
    #     for i in range(iterations):
    #         output = modelCase.G(x_val)
    #         if is_first:
    #             outputs = output
    #             is_first = False
    #         else:
    #             outputs = np.concatenate((outputs,output), axis=0)
    #     print("Inference is done")
    #     return outputs


In [145]:
target_shape

[96, 128, 1]

In [146]:
input_shape

[96, 128, 2]

In [ ]:
modelCase = WGANModel(mode="train", hparams_dict=hparams_dict,
                      input_shape=input_shape,target_shape=target_shape,
                      discriminator=discriminator, generator=generator)

modelCase.make_data_generator(ds_train, ds_val, ds_test)
modelCase.train(n_samples)

You are training both generator and discriminator (WGAN)
You are training 5 times more discriminator/critic for one time generator
[0/1830] D_loss=-0.36460 G_loss=0.49876, g_recon_loss=79816254.64380 training time per step: 1.78149/s
[5/1830] D_loss=-0.29866 G_loss=0.54312, g_recon_loss=64123080.65329 training time per step: 1.74846/s
[10/1830] D_loss=-0.03419 G_loss=0.38774, g_recon_loss=61527745.91914 training time per step: 1.67509/s
[15/1830] D_loss=-0.01310 G_loss=0.39193, g_recon_loss=86331556.26000 training time per step: 1.59867/s
[20/1830] D_loss=0.01324 G_loss=0.58556, g_recon_loss=59381456.60832 training time per step: 1.58591/s
[25/1830] D_loss=-0.13854 G_loss=0.37832, g_recon_loss=97644509.98135 training time per step: 1.60391/s
[30/1830] D_loss=0.03697 G_loss=0.58469, g_recon_loss=59250733.93042 training time per step: 1.64778/s
[35/1830] D_loss=0.13463 G_loss=0.40334, g_recon_loss=51105119.81785 training time per step: 1.53365/s
[40/1830] D_loss=0.20547 G_loss=0.40955, g

In [ ]:
ds_train.shape

In [ ]:
input_shape